# Linear regression 

In this exercise you will use linear regression to predict flat prices. One more time, training will be handled via gradient descent. Although, contratry to the first exercise, we will now:
* have multiple features (i.e. variables used to make the prediction),
* employ some basic feature engineering,
* work with a different loss function.

Let's start with getting the data.

In [ ]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

--2021-03-11 23:45:05--  https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/zey0gx91pna8irj/mieszkania.csv [following]
--2021-03-11 23:45:06--  https://www.dropbox.com/s/dl/zey0gx91pna8irj/mieszkania.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9c9db9cd394b64629fc838db70.dl.dropboxusercontent.com/cd/0/get/BKg6MJoa95C7F4SKqNaOa029X5eJsNk1Xy61dryd_bm6qQEfVexSYQnRmO1x7ba22v668E1z-9lbjNZG_EbX46U1YqIyzbJr38Ist5SgnQYQqlpEsNQOP0VYz_1V_lqUS7SUhmyxGEQDSnXF6kWbUFN8/file?dl=1# [following]
--2021-03-11 23:45:06--  https://uc9c9db9cd394b64629fc838db70.dl.dropboxusercontent.com/cd/0/get/BKg6MJoa95C7F4SKqNaOa029X5eJsNk1Xy61dryd_bm6qQEfVexSYQnRmO1x7ba22v668E1z-9lbjNZG_E

In [ ]:
!head mieszkania.csv mieszkania_test.csv

==> mieszkania.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
104,mokotowo,2,2,1940,1,780094
43,ochotowo,1,1,1970,1,346912
128,grodziskowo,3,2,1916,1,523466
112,mokotowo,3,2,1920,1,830965
149,mokotowo,3,3,1977,0,1090479
80,ochotowo,2,2,1937,0,599060
58,ochotowo,2,1,1922,0,463639
23,ochotowo,1,1,1929,0,166785
40,mokotowo,1,1,1973,0,318849

==> mieszkania_test.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
71,wolowo,2,2,1912,1,322227
45,mokotowo,1,1,1938,0,295878
38,mokotowo,1,1,1999,1,306530
70,ochotowo,2,2,1980,1,553641
136,mokotowo,3,2,1939,1,985348
128,wolowo,3,2,1983,1,695726
23,grodziskowo,1,1,1975,0,99751
117,mokotowo,3,2,1942,0,891261
65,ochotowo,2,1,2002,1,536499


In [ ]:
import pandas as pd
train = pd.read_csv('mieszkania.csv')
test = pd.read_csv('mieszkania_test.csv')

total = pd.concat([train,test])

total = pd.get_dummies(total, prefix='dzielnica')

train = total[:len(train)]
test = total[len(train):]

Each row in the data represents a separate property. Our goal is to use the data from `mieszkania.csv` to create a model that can predict a property's price (i.e. `cena`) given its features (i.e. `m2,dzielnica,ilosc_sypialni,ilosc_lazienek,rok_budowy,parking_podziemny`). 

From now on, we should interfere only with `mieszkania.csv` (dubbed the training dataset) to make our decisions and create the model. The (only) purpose of `mieszkania_test.csv` is to test our model on **unseen** data.

Our predictions should minimize the so-called mean squared logarithmic error:
$$
MSLE = \frac{1}{n} \sum_{i=1}^n (\log(1+y_i) - \log(1+p_i))^2,
$$
where $y_i$ is the ground truth, and $p_i$ is our prediction.

Let's start with implementing the loss function.

In [ ]:
import numpy as np

In [ ]:
def msle(ys, ps):
    assert len(ys) == len(ps)
    return np.mean((np.log1p(ys)-np.log1p(ps))**2)

The simplest model is predicting the same constant for each instance. Test your implementation of msle against outputing the mean price.

In [ ]:
###################################################
# Compute msle for outputing the mean price #
###################################################
n=len(test)

mean_price_train = np.mean(train['cena'])
print(f'mean price {mean_price_train}')
print(f"msle for the same constant: {msle(ys=test['cena'], ps=np.full(n, fill_value=mean_price_train))}")

mean price 507919.49
msle for the same constant: 0.4284115392580848


Recall that outputing the mean minimzes $MSE$. However, we're now dealing with $MSLE$.

Think of a constant that should result in the lowest $MSLE$.

In [ ]:
#############################################
# TODO: Find this constant and compute msle #
#############################################
# \sum_{i=1}^n (log(1+yi) - log(1+c))^2 ---> min
# when log(1+c) = 1/n (log(1+y1)+...+log(1+yn))
# 1+c = np.exp(1/n (log(1+y1)+...+log(1+yn)) )
# c = np.exp(np.mean(np.log(1+train['cena']))) - 1
c = np.exp(np.mean(np.log(1+train['cena']))) - 1
print(f"msle for the same constant: {msle(ys=test['cena'], ps=np.full(n, fill_value=np.log(1+c)))}")

msle for the same constant: 0.4275752033760174


Now, let's implement a standard linear regression model. 

In [ ]:
def generate(train, test):
  train_y = train['cena'].to_numpy()
  test_y = test['cena'].to_numpy()
  n=len(test_y)

  train_x = train.drop('cena', axis = 1)
  test_x = test.drop('cena', axis = 1)
  
  mean, std = train_x.mean().to_numpy(), train_x.std().to_numpy()

  train_x, test_x = train_x.to_numpy(), test_x.to_numpy()

  train_x = (train_x - mean) / std
  train_x = np.hstack((np.ones_like(train_x), train_x)) # bias

  test_x = (test_x - mean) / std
  test_x = np.hstack((np.ones_like(test_x), test_x)) # bias
  return train_x, train_y, test_x, test_y, n

  
train_x, train_y, test_x, test_y, n = generate(train, test)

In [ ]:
##########################################################
# TODO: Implement linear regression and compute its msle #
##########################################################
lr=0.001

def evaluate(ws, xs, ys):
    ps = np.dot(xs, ws)
    assert len(ys) == len(ps)
    return np.mean((ys - ps)**2)
    

def fit(x, y, steps):
  print(x.shape[1])
  W = np.random.normal(size=x.shape[1])
  for i in range(steps):
    # Loss(w0, w1, w2,..., wn) = (y - \sum w_ix_i)**2
    # pred = x1w1 + ... + xnwn
    preds = np.dot(x, W)
    delta = preds - y

    # Loss =  \sum (yt - (x1w1 + ... + xnwn))^2
    # dLoss/dw_i = 2*(w1x1 + ... +wnxn - y)*xi
    gradient = (2/n)*np.dot(x.T,  delta)
    W-=lr*gradient
    
    if i % 10_000 == 0:
      loss=evaluate(W, x, y)
      print('Iter: {:>3} Loss: {:8.8f}'.format(i, loss))
  return W



In [ ]:
W=fit(train_x, train_y, 100000)
W_l=fit(train_x, np.log1p(train_y), 100000)

18
Iter:   0 Loss: 321879379890.82055664
Iter: 10000 Loss: 4158723999.80191660
Iter: 20000 Loss: 3910094975.47649288
Iter: 30000 Loss: 3874334353.46551132
Iter: 40000 Loss: 3869190065.90122557
Iter: 50000 Loss: 3868450042.48553467
Iter: 60000 Loss: 3868343587.57822609
Iter: 70000 Loss: 3868328273.67457294
Iter: 80000 Loss: 3868326070.71699142
Iter: 90000 Loss: 3868325753.81398439
18
Iter:   0 Loss: 270.81180309
Iter: 10000 Loss: 0.03621108
Iter: 20000 Loss: 0.03110223
Iter: 30000 Loss: 0.03036959
Iter: 40000 Loss: 0.03026420
Iter: 50000 Loss: 0.03024904
Iter: 60000 Loss: 0.03024686
Iter: 70000 Loss: 0.03024655
Iter: 80000 Loss: 0.03024650
Iter: 90000 Loss: 0.03024649


In [ ]:
pred=test_x @ W
print(f'Score after regression = {msle(test_y, pred)}')
print(list(zip(pred[:10], test_y[:10])))

Score after regression = 0.18097954619040799
[(377531.00733555085, 322227), (371324.0075861293, 295878), (363304.24032035936, 306530), (552899.7397513028, 553641), (961779.6817848356, 985348), (757216.174596213, 695726), (1711.421480350571, 99751), (814211.6758486971, 891261), (538791.6709008679, 536499), (637412.9729520866, 527093)]


Note that the loss function that the algorithms optimizes (i.e $MSE$) differs from $MSLE$. We've already seen that this may result in a suboptimal solution.

How can you change the setting so that we optimze $MSLE$ instead?

Hint: 
<sub><sup><sub><sup><sub><sup>
Be lazy. We don't want to change the algorithm.
</sup></sub></sup></sub></sup></sub>

In [ ]:
pred=np.expm1(test_x @ W_l)
print(f'Score after regression = {msle(test_y, pred)}')
print(list(zip(pred[:10], test_y[:10])))


Score after regression = 0.03656891333674678
[(325345.70468483336, 322227), (319964.2647759138, 295878), (296896.2926081494, 306530), (424585.85935797397, 553641), (1198592.430718195, 985348), (801487.8472737543, 695726), (141146.08215618547, 99751), (860737.8811225289, 891261), (447010.2152104922, 536499), (618671.4825228279, 527093)]


Without any feature engineering our model approximates the price as a linear combination of original features:
$$
\text{price} \approx w_1 \cdot \text{area} + w_2 \cdot \text{district} + \dots.
$$
Let's now introduce some interactions between the variables. For instance, let's consider a following formula:
$$
\text{price} \approx w_1 \cdot \text{area} \cdot \text{avg. price in the district per sq. meter} + w_2 \cdot \dots + \dots.
$$
Here, we model the price with far greater granularity, and we may expect to see more acurate results.

Add some feature engineering to your model. Be sure to play with the data and not with the algorithm's code. 

Think how to make sure that your model is capable of capturing the $w_1 \cdot \text{area} \cdot \text{avg. price...}$ part, without actually computing the averages.

*Hint*: 
Is having a binary encoding for each district and multiplying it by area enough?


*Hint* 2: 
Why not multiply everything together? I.e. (A,B,C) -> (AB,AC,BC).


In [ ]:
###############################################
# TODO: Implement the feature engieering part #
###############################################

# m2,dzielnica,ilosc_sypialni,ilosc_lazienek,rok_budowy,parking_podziemny

cols = [x for x in train.columns if x != 'cena']

def mul_cols(df):
  a,b = df.shape
  ndf = np.zeros(shape=(a,int(b*(b+1)/2)))
  k=0
  for i in range(b):
    for j in range(i, b):
      ndf[:, k] = df[:, i]*df[:, j]
      k+=1
  return ndf

train_x_new = mul_cols(train_x)
test_x_new = mul_cols(test_x)

In [ ]:
W_l=fit(train_x_new, np.log1p(train_y), 100000)
pred=np.expm1(test_x_new @ W_l)
print(f'Score after regression = {msle(test_y, pred)}')
print(list(zip(pred[:10], test_y[:10])))


171
Iter:   0 Loss: 166.94156413
Iter: 10000 Loss: 0.09855877
Iter: 20000 Loss: 0.01731904
Iter: 30000 Loss: 0.00981513
Iter: 40000 Loss: 0.00870900
Iter: 50000 Loss: 0.00827872
Iter: 60000 Loss: 0.00798367
Iter: 70000 Loss: 0.00775464
Iter: 80000 Loss: 0.00757216
Iter: 90000 Loss: 0.00742492
Score after regression = 0.01973913352438987
[(433021.6121943501, 322227), (339732.0189798612, 295878), (278820.59103135654, 306530), (523638.981215183, 553641), (949488.4796265454, 985348), (654759.3424452044, 695726), (118014.02668216478, 99751), (866570.7466566951, 891261), (533889.0250964419, 536499), (502419.238145803, 527093)]
